# Kerakli kutubxonalarni chaqiramiz

In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()
labeller = LabelEncoder()

# dataset ni yuklaymiz

In [6]:
df = pd.read_csv(r"C:\Users\bunyo\OneDrive\Desktop\AI_Course\ModularProgramProjects\FirstWeekProject\data\raw_data\wind_dataset.csv")

In [7]:
df.sample(3)

,DATE,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G
300,1961-10-28,9.75,0,0.0,0.0,11.7,0.0,3.7,1.0
4610,1973-08-16,4.21,0,0.0,0.0,24.2,0.0,13.2,8.0
1083,1963-12-20,8.38,0,0.0,4.0,NaN,4.0,NaN,-8.3


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   DATE     6574 non-null   object 
 1   WIND     6574 non-null   float64
 2   IND      6574 non-null   int64  
 3   RAIN     6574 non-null   float64
 4   IND.1    6513 non-null   float64
 5   T.MAX    5953 non-null   float64
 6   IND.2    6513 non-null   float64
 7   T.MIN    5900 non-null   float64
 8   T.MIN.G  6214 non-null   float64
dtypes: float64(7), int64(1), object(1)
memory usage: 462.4+ KB


# DATE ustuni bilan ishlaymiz

In [10]:
df['DATE'] = pd.to_datetime(df['DATE'])

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   DATE     6574 non-null   datetime64[ns]
 1   WIND     6574 non-null   float64       
 2   IND      6574 non-null   int64         
 3   RAIN     6574 non-null   float64       
 4   IND.1    6513 non-null   float64       
 5   T.MAX    5953 non-null   float64       
 6   IND.2    6513 non-null   float64       
 7   T.MIN    5900 non-null   float64       
 8   T.MIN.G  6214 non-null   float64       
dtypes: datetime64[ns](1), float64(7), int64(1)
memory usage: 462.4 KB


# DATE ustunini oddiy yil oy va kunlar ga ozgartirdik

In [13]:
df['Year'] = df['DATE'].dt.year
df['Month'] = df['DATE'].dt.month
df['Day'] = df['DATE'].dt.day

In [15]:
df.head()

,DATE,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G,Year,Month,Day
0,1961-01-01,13.67,0,0.2,0.0,9.5,0.0,3.7,-1.0,1961,1,1
1,1961-01-02,11.50,0,5.1,0.0,7.2,0.0,4.2,1.1,1961,1,2
2,1961-01-03,11.25,0,0.4,0.0,5.5,0.0,0.5,-0.5,1961,1,3
3,1961-01-04,8.63,0,0.2,0.0,5.6,0.0,0.4,-3.2,1961,1,4
4,1961-01-05,11.92,0,10.4,0.0,7.2,1.0,-1.5,-7.5,1961,1,5


In [16]:
df.isnull().sum()

DATE         0
WIND         0
IND          0
RAIN         0
IND.1       61
T.MAX      621
IND.2       61
T.MIN      674
T.MIN.G    360
Year         0
Month        0
Day          0
dtype: int64

In [23]:
df = df.drop('DATE', axis=1)

# feature engineering qilish

In [24]:
df['T_AVG'] = (df['T.MAX'] - df['T.MIN']) / 2
# Bu xolatda ortacha xavo xarorati chiqarib olindi

df['IS_RAINIY'] = (df['RAIN'] > 0).astype(int)
# Bu code orqali aniq yomgir yog'ganligi bilib olamiz (1 bolsa yomgir bor. 0 bolsa yomgir yoq)

df['IS_FROST'] = (df['T.MIN.G'] < 0).astype(int)
# Bunda yer muzlagan yoki yoq ligini bilamiz (1 bolsa muzlagan 0 bolsa muzlamagan)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WIND       6574 non-null   float64
 1   IND        6574 non-null   float64
 2   RAIN       6574 non-null   float64
 3   IND.1      6574 non-null   float64
 4   T.MAX      6574 non-null   float64
 5   IND.2      6574 non-null   float64
 6   T.MIN      6574 non-null   float64
 7   T.MIN.G    6574 non-null   float64
 8   Year       6574 non-null   float64
 9   Month      6574 non-null   float64
 10  Day        6574 non-null   float64
 11  T_AVG      6574 non-null   float64
 12  IS_RAINIY  6574 non-null   int64  
 13  IS_FROST   6574 non-null   int64  
dtypes: float64(12), int64(2)
memory usage: 719.2 KB


# Class orqli data ga ishlov beramiz

In [26]:
class Preprocessing:
    def __init__(self,df):
        self.df = df
        
# Encoding qiluvchi
    def encoding_qilish(self):
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                if self.df[col].nunique() <= 3:
                    new_df = pd.get_dummies(self.df[col], prefix='New', dtype=int)
                    self.df.drop(columns=[col],inplace=True)
                    self.df = pd.concat([self.df,new_df],axis=1)
                else:
                    self.df[col] = labeller.fit_transform(self.df[col])
        return self
    

# To'ldruvchi
    def fillingNan(self):
        for cols in self.df.columns:
            if self.df[cols].isnull().any():
                if self.df[cols].dtype == 'object':
                    self.df[cols].fillna(self.df[cols].mode()[0], inplace=True)
                else:
                    self.df[cols].fillna(self.df[cols].mean(),inplace=True)
        return self
    
    
# Scale qiluvchi
    def scaling_qilish(self): 
        for cols in self.df.columns: 
            if self.df[cols].dtype != 'object' and self.df[cols].name != 'WIND': 
                self.df[cols] = minmax.fit_transform(self.df[[cols]]) 
        return self.df


In [27]:
pre_processing = Preprocessing(df)
df1 = pre_processing.fillingNan().encoding_qilish().scaling_qilish()

In [28]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6574 entries, 0 to 6573
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   WIND       6574 non-null   float64
 1   IND        6574 non-null   float64
 2   RAIN       6574 non-null   float64
 3   IND.1      6574 non-null   float64
 4   T.MAX      6574 non-null   float64
 5   IND.2      6574 non-null   float64
 6   T.MIN      6574 non-null   float64
 7   T.MIN.G    6574 non-null   float64
 8   Year       6574 non-null   float64
 9   Month      6574 non-null   float64
 10  Day        6574 non-null   float64
 11  T_AVG      6574 non-null   float64
 12  IS_RAINIY  6574 non-null   float64
 13  IS_FROST   6574 non-null   float64
dtypes: float64(14)
memory usage: 719.2 KB


In [29]:
df1.head()

,WIND,IND,RAIN,IND.1,T.MAX,IND.2,T.MIN,T.MIN.G,Year,Month,Day,T_AVG,IS_RAINIY,IS_FROST
0,13.67,0.0,0.002985,0.0,0.356877,0.00,0.515254,0.443709,0.0,0.0,0.000000,0.439104,1.0,0.0
1,11.50,0.0,0.076119,0.0,0.271375,0.00,0.532203,0.513245,0.0,0.0,0.033333,0.320144,1.0,0.0
2,11.25,0.0,0.005970,0.0,0.208178,0.00,0.406780,0.460265,0.0,0.0,0.066667,0.392398,1.0,0.0
3,8.63,0.0,0.002985,0.0,0.211896,0.00,0.403390,0.370861,0.0,0.0,0.100000,0.400650,1.0,0.0
4,11.92,0.0,0.155224,0.0,0.271375,0.25,0.338983,0.228477,0.0,0.0,0.133333,0.544500,1.0,0.0


# preprocessed data ni saqlaymiz

In [30]:
folder = r"C:\Users\bunyo\OneDrive\Desktop\AI_Course\ModularProgramProjects\FirstWeekProject\data\preprocessed_data"
path = os.path.join(folder,'preprocessed_data.csv')
df1.to_csv(path, index=False)